In [1]:
import pandas as pd
import os
from pathlib import Path
import re

In [2]:
neighborhood_tracts = {
    'Fenway':['9815.01', '101.03', '101.04', '102.04','104.04', '104.05', '102.03', '104.08', '104.03'],
    'Roxbury':['815', '906','804.01','806.01','9803','814','818','801','820','813','819','817','803','805','821','904'],
    'Back Bay':['106','107.01','107.02','108.01','108.02','105'],
    'South End':['704.02','711.01','712.01','703','705','708','706','707','709']
}

In [3]:
tract_lookup_table = pd.DataFrame(columns = ['Tract','Neighborhood'])
for hood in list(neighborhood_tracts.keys()):
    for tract in neighborhood_tracts[hood]:
        row = {'Tract': tract, 'Neighborhood': hood}
        tract_lookup_table = tract_lookup_table.append(row, ignore_index=True)
tract_lookup = tract_lookup_table.set_index('Tract').to_dict(orient='index')

In [4]:
def lookup(tract):
    tract_no = tract.split( )[2]
    return tract_lookup[tract_no]['Neighborhood']

In [5]:
def get_paths(subset):
    tracts = []
    paths = []
    for path, subdirs, files in os.walk(Path(f'data/{subset}')):
        tracts = subdirs
        for name in files:
            file_path = os.path.join(path,name)
            if name[0] != '.':
                paths.append(file_path)
    return paths

In [17]:
paths = get_paths('Mass Ave Census Tracts')
paths[0]

'data/Mass Ave Census Tracts/708/Household_Income_in_the_Past_12_Months_(In_2019_Inflation-adjusted_Dollars)_5YR_ACS_2015-2019.csv'

In [19]:
def parse(file, bigDF, totalDF):
    df = pd.read_csv(file)
    tract, county, state = (list(df.columns)[1]).split(',')
    #tract_no = tract.split( )[2]
    tract_no = tract
    measure = df.iloc[0][list(df.columns)[0]]
    for i in range(1,df.index.stop-2):
        category = df.iloc[i][list(df.columns)[0]]
        value = df.iloc[i][list(df.columns)[1]]
        row = {
            'Neighborhood': lookup(tract_no),
            'Tract':tract_no,
            'Measure':measure,
            'Category':category,
            'Value':value
        }
        print(row)
        if row['Category'].lower()=='total':
            totalDF = totalDF.append(row, ignore_index=True)
        else:
            bigDF = bigDF.append(row, ignore_index=True)
    return bigDF, totalDF

In [23]:
pd.read_csv(paths[0])

,Tracts,"Census Tract 708, Suffolk County, Massachusetts"
0,Household Income,Count
1,"Less than $14,999",323
2,"$15,000 - $24,999",124
3,"$25,000 - $34,999",60
4,"$35,000 - $49,999",95
5,"$50,000 - $74,999",332
6,"$75,000 - $99,999",110
7,"$99,999 - $149,999",192
8,"$150,000 or more",793
9,Total,2029


In [7]:
def summarize(measure, bigDF, totalDF):
    try:
        os.mkdir(f'data/summaries')
    except:
        pass
    measure = measure.replace('/','-')
    measureDF = bigDF[bigDF['Measure'] == measure].drop(columns=['Measure'])
    measureDF.to_csv(f'data/summaries/{measure}_joined.csv')
    measuretotalDF = totalDF[totalDF['Measure'] == measure].drop(columns=['Measure', 'Category'])
    measuretotalDF.to_csv(f'data/summaries/{measure}_totals.csv')

In [20]:
joinDF = pd.DataFrame(columns=[
    'Neighborhood',
    'Tract',
    'Measure',
    'Category',
    'Value'])
totalDF = joinDF.copy()
paths = get_paths('Mass Ave Census Tracts')
for file in paths:
    joinDF, totalDF = parse(file, joinDF, totalDF)
    joinDF.to_csv(f'data/data_joined.csv')
    totalDF.to_csv(f'data/data_totals.csv')
measures = list(totalDF['Measure'])
for measure in measures:
    summarize(measure, joinDF, totalDF)

{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Household Income', 'Category': 'Less than $14,999', 'Value': '323'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Household Income', 'Category': '$15,000 - $24,999', 'Value': '124'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Household Income', 'Category': '$25,000 - $34,999', 'Value': '60'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Household Income', 'Category': '$35,000 - $49,999', 'Value': '95'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Household Income', 'Category': '$50,000 - $74,999', 'Value': '332'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Household Income', 'Category': '$75,000 - $99,999', 'Value': '110'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Household Income', 'Category': '$99,999 - $149,999', 'Value': '192'}
{'Neighborhood': 'South End'

{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Resident Employment by Occupation', 'Category': 'Sales, office and administrative support', 'Value': '514'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Resident Employment by Occupation', 'Category': 'Natural resources, construction, and maintenance', 'Value': '0'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Resident Employment by Occupation', 'Category': 'Production, transportation, and material moving', 'Value': '64'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Resident Employment by Occupation', 'Category': 'Total', 'Value': '2640'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Nativity', 'Category': 'Native', 'Value': '3083'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Nativity', 'Category': 'Naturalized citizen', 'Value': '345'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract

{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'Age', 'Category': 'Total', 'Value': '3597'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'School Level', 'Category': 'Enrolled in nursery school preschool', 'Value': '30'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'School Level', 'Category': 'Enrolled in kindergarten', 'Value': '14'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'School Level', 'Category': 'Enrolled in grade 1 to grade 4', 'Value': '34'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'School Level', 'Category': 'Enrolled in grade 5 to grade 8', 'Value': '40'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'School Level', 'Category': 'Enrolled in grade 9 to grade 12', 'Value': '0'}
{'Neighborhood': 'South End', 'Tract': 'Census Tract 708', 'Measure': 'School Level', 'Category': 'Enrolled in college undergraduate years', 'Value': 

{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Bedrooms', 'Category': 'No bedroom', 'Value': '$1479'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Bedrooms', 'Category': '1 bedroom', 'Value': '$2278'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Bedrooms', 'Category': '2 bedrooms', 'Value': '$1956'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Bedrooms', 'Category': '3 bedrooms', 'Value': '$2583'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Bedrooms', 'Category': 'Total', 'Value': '$2,120'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Unit in Structure', 'Category': '1 detached', 'Value': '13'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Unit in Structure', 'Category': '1 attached', 'Value': '36'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Unit in Structure', 'Category': '2', 'Value': '3

{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Resident Employment by Industry', 'Category': 'Transportation and warehousing', 'Value': '0'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Resident Employment by Industry', 'Category': 'Information', 'Value': '57'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Resident Employment by Industry', 'Category': 'Finance and insurance', 'Value': '110'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Resident Employment by Industry', 'Category': 'Real estate and rental and leasing', 'Value': '57'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Resident Employment by Industry', 'Category': 'Professional, scientific, and technical services', 'Value': '446'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 105', 'Measure': 'Resident Employment by Industry', 'Category': 'Management of companies and enterprises', 'Value': '0'}
{'Neigh

...

{'Neighborhood': 'Fenway', 'Tract': 'Census Tract 104.03', 'Measure': 'Year Structure Built', 'Category': 'Built 1940 to 1949', 'Value': '52'}
{'Neighborhood': 'Fenway', 'Tract': 'Census Tract 104.03', 'Measure': 'Year Structure Built', 'Category': 'Built 1939 or earlier', 'Value': '587'}
{'Neighborhood': 'Fenway', 'Tract': 'Census Tract 104.03', 'Measure': 'Year Structure Built', 'Category': 'Total', 'Value': '2076'}
{'Neighborhood': 'Fenway', 'Tract': 'Census Tract 104.03', 'Measure': 'Housing Tenure', 'Category': 'Studio or 1 bedroom', 'Value': '52'}
{'Neighborhood': 'Fenway', 'Tract': 'Census Tract 104.03', 'Measure': 'Housing Tenure', 'Category': '2 bedrooms', 'Value': '29'}
{'Neighborhood': 'Fenway', 'Tract': 'Census Tract 104.03', 'Measure': 'Housing Tenure', 'Category': '3 or more bedrooms', 'Value': '8'}
{'Neighborhood': 'Fenway', 'Tract': 'Census Tract 104.03', 'Measure': 'Housing Tenure', 'Category': 'Total', 'Value': '89'}
{'Neighborhood': 'Fenway', 'Tract': 'Census Tract 1

{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': "Field of Bachelor's Degree for First Major", 'Category': 'Education', 'Value': '33'}
{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': "Field of Bachelor's Degree for First Major", 'Category': 'Literature and Languages', 'Value': '9'}
{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': "Field of Bachelor's Degree for First Major", 'Category': 'Liberal Arts and History', 'Value': '9'}
{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': "Field of Bachelor's Degree for First Major", 'Category': 'Visual and Performing Arts', 'Value': '30'}
{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': "Field of Bachelor's Degree for First Major", 'Category': 'Communications', 'Value': '15'}
{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': "Field of Bachelor's Degree for First Major", 'Category': 'Others', 'Value': '69'}
{'Neighborhood': 'Roxbury', '

{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': 'Resident Employment by Industry', 'Category': 'Agriculture, forestry, fishing and hunting, and mining', 'Value': '0'}
{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': 'Resident Employment by Industry', 'Category': 'Utilities', 'Value': '0'}
{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': 'Resident Employment by Industry', 'Category': 'Construction', 'Value': '19'}
{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': 'Resident Employment by Industry', 'Category': 'Manufacturing', 'Value': '19'}
{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': 'Resident Employment by Industry', 'Category': 'Wholesale trade', 'Value': '0'}
{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': 'Resident Employment by Industry', 'Category': 'Retail trade', 'Value': '43'}
{'Neighborhood': 'Roxbury', 'Tract': 'Census Tract 801', 'Measure': 'Resident Employ

{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': "Field of Bachelor's Degree for First Major", 'Category': 'Computers, Mathematics and Statistics', 'Value': '92'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': "Field of Bachelor's Degree for First Major", 'Category': 'Biological, Agricultural, and Environmental Sciences', 'Value': '69'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': "Field of Bachelor's Degree for First Major", 'Category': 'Physical and Related Sciences', 'Value': '63'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': "Field of Bachelor's Degree for First Major", 'Category': 'Psychology', 'Value': '116'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': "Field of Bachelor's Degree for First Major", 'Category': 'Social Sciences', 'Value': '370'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': "Field of Bachelor's Degree for First 

{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': nan, 'Category': 'Median household income', 'Value': '$117,875'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': 'Age', 'Category': 'Under 18 years', 'Value': '0'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': 'Age', 'Category': '18 to 64 years', 'Value': '8.382'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': 'Age', 'Category': '65 years and over', 'Value': '3.658'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': 'Age', 'Category': 'Total', 'Value': '7.306'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': 'Resident Employment by Industry', 'Category': 'Agriculture, forestry, fishing and hunting, and mining', 'Value': '0'}
{'Neighborhood': 'Back Bay', 'Tract': 'Census Tract 107.01', 'Measure': 'Resident Employment by Industry', 'Category': 'Utilities', 'Value': '0'}
{'Neighborhood': 'Back Bay',

In [22]:
totalDF

,Neighborhood,Tract,Measure,Category,Value
0,South End,Census Tract 708,Household Income,Total,2029
1,South End,Census Tract 708,Health Insurance,Total,3597
2,South End,Census Tract 708,Field of Bachelor's Degree for First Major,Total,2386
3,South End,Census Tract 708,Household Size,Total,631
4,South End,Census Tract 708,Year Structure Built,Total,2201
...,...,...,...,...,...
250,Back Bay,Census Tract 107.01,Resident Employment by Industry,Total,1767
251,Back Bay,Census Tract 107.01,Race/Ethnicity,Total,2383
252,Back Bay,Census Tract 107.01,Gender,Total,2383
253,Back Bay,Census Tract 107.01,Age,Total,2383


In [9]:
def convert_numeric(v): return re.sub("[^0-9]", "", v)

In [10]:
joinDF['Value'] = [float(convert_numeric(v)) if isinstance(v, str) else v for v in joinDF['Value']]
grouped = joinDF.drop(columns='Tract').groupby(['Neighborhood','Measure','Category']).sum()
grouped

Value
Neighborhood Measure              Category                    
Back Bay     Age                  0-4                    259.0
                                  18 to 64 years       20262.0
                                  18-24                 1974.0
                                  25-34                 2973.0
                                  35-44                 1079.0
...                                                        ...
South End    Year Structure Built Built 1980 to 1989      90.0
                                  Built 1990 to 1999      71.0
                                  Built 2000 to 2009     148.0
                                  Built 2010 to 2013       8.0
                                  Built 2014 or later      0.0

[616 rows x 1 columns]

In [11]:
"""
def get_neighborhood_tracts():
    neighborhoods = []
    tract_dict = {}
    for path, subdirs, files in os.walk(Path('data/Mass Ave Neighborhoods')):
        neighborhoods = subdirs
        break
    for hood in neighborhoods:
        for path, subdirs, files in os.walk(Path('data/Mass Ave Neighborhoods')):
            try:
                tracts = []
                df = pd.read_csv(os.path.join(path,files[0]))
                tract_strings = list(df.columns[2:])
                for t in tract_strings:
                    tract = (t.split(',')[0]).split(' ')[-1]
                    tracts.append(tract)
            except:
                pass
            tract_dict[hood]=list(set(tracts))
    return tract_dict
"""

"\ndef get_neighborhood_tracts():\n    neighborhoods = []\n    tract_dict = {}\n    for path, subdirs, files in os.walk(Path('data/Mass Ave Neighborhoods')):\n        neighborhoods = subdirs\n        break\n    for hood in neighborhoods:\n        for path, subdirs, files in os.walk(Path('data/Mass Ave Neighborhoods')):\n            try:\n                tracts = []\n                df = pd.read_csv(os.path.join(path,files[0]))\n                tract_strings = list(df.columns[2:])\n                for t in tract_strings:\n                    tract = (t.split(',')[0]).split(' ')[-1]\n                    tracts.append(tract)\n            except:\n                pass\n            tract_dict[hood]=list(set(tracts))\n    return tract_dict\n"